In [ ]:
# <xaiArtifact artifact_id="7fca53d3-eeb9-4e51-9992-096280baf133" title="test_sentiment_model.ipynb" contentType="text/x-python">
# Install required packages
!pip install transformers datasets pandas numpy torch scikit-learn

import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# Load and preprocess test data
def load_test_data(file_path):
    df = pd.read_csv(file_path)
    df = df[df['sentiment'] != 'Irrelevant']
    sentiment_map = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
    df['label'] = df['sentiment'].map(sentiment_map)
    return df

# Evaluate model
def evaluate_model(test_df, model_path="sentiment_model"):
    tokenizer = RobertaTokenizer.from_pretrained(model_path)
    model = RobertaForSequenceClassification.from_pretrained(model_path)
    
    predictions = []
    true_labels = []
    
    for _, row in test_df.iterrows():
        inputs = tokenizer(row['Tweet content'], return_tensors="pt", truncation=True, max_length=128)
        with torch.no_grad():
            logits = model(**inputs).logits
        predicted_class = torch.argmax(logits, dim=1).item()
        predictions.append(predicted_class)
        true_labels.append(row['label'])
    
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average='weighted')
    
    return accuracy, f1

# Example usage
if __name__ == "__main__":
    # Load test data (replace with your file path)
    test_df = load_test_data("test_dataset.csv")
    
    # Evaluate model
    accuracy, f1 = evaluate_model(test_df)
    
    print(f"Test Accuracy: {accuracy:.2%}")
    print(f"Test F1 Score: {f1:.2%}")